In [ ]:
# %%bash

# wget https://raw.githubusercontent.com/brevdev/notebooks/main/assets/setup-ngc.sh -O setup-ngc
# chmod +x setup-ngc
# ./setup-ngc

In [ ]:
# this should the .nemo checkpoint that is saved
!ls ./llama-3_1-8b-instruct-nemo

In [38]:
import os
import json
import numpy as np
import transformers
import torch
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split

In [39]:
os.environ["WANDB_DISABLED"] = "false"

In [40]:
dataset = load_dataset("b-mc2/sql-create-context", split="train")

In [41]:
df = dataset.to_pandas()

# Split into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

# Split the training set into training and validation sets
train_df, val_df = train_test_split(train_df, test_size=0.005, random_state=42)

# Convert back to Dataset objects
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

print(f"Train size: {len(train_dataset)}, Validation size: {len(val_dataset)}, Test size: {len(test_dataset)}")

Train size: 70365, Validation size: 354, Test size: 7858


In [42]:
print(train_dataset[24])

{'answer': 'SELECT ground FROM table_name_83 WHERE competition = "tim trophy" AND time = "23:00 cet"', 'question': 'What is the ground of the tim trophy competition, which had a time of 23:00 cet?', 'context': 'CREATE TABLE table_name_83 (ground VARCHAR, competition VARCHAR, time VARCHAR)', '__index_level_0__': 68005}


In [43]:
def generate_input_output(data):

    data_json = PROMPT =f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables."""
    
    new_data = {
                "input": f'''{PROMPT} \nQUESTION: {data["question"]} \nCONTEXT: {data["context"]}''',
                "output": data['answer']
            }
    return new_data

In [44]:
def create_jsonl_files(input_dataset, output_file_name):
    for index in range(input_dataset.num_rows):
        new_data = generate_input_output(input_dataset[index])
        with open(output_file_name, 'a') as outfile:
            json.dump(new_data, outfile)
            outfile.write('\n')
    return 1

In [45]:
create_jsonl_files(train_dataset, 'train_dataset_preprocessed.jsonl')

1

In [46]:
create_jsonl_files(test_dataset, 'test_dataset_preprocessed.jsonl')

1

In [47]:
create_jsonl_files(val_dataset, 'val_dataset_preprocessed.jsonl')

1

In [48]:
%%bash

# Set paths to the model, train, validation and test sets.
MODEL="./llama-3_1-8b-instruct-nemo_v1.0/llama3_1_8b_instruct.nemo"

TRAIN_DS="[train_dataset_preprocessed.jsonl]"
VALID_DS="[val_dataset_preprocessed.jsonl]"
TEST_DS="[test_dataset_preprocessed.jsonl]"
TEST_NAMES="[n2sql]"

SCHEME="lora"
TP_SIZE=1
PP_SIZE=1

rm -rf results
OUTPUT_DIR="./results/Meta-llama3.1-8B-Instruct-titlegen"

torchrun --nproc_per_node=1 \
/opt/NeMo/examples/nlp/language_modeling/tuning/megatron_gpt_finetuning.py \
    exp_manager.exp_dir=${OUTPUT_DIR} \
    exp_manager.explicit_log_dir=${OUTPUT_DIR} \
    trainer.devices=1 \
    trainer.num_nodes=1 \
    trainer.precision=bf16-mixed \
    trainer.val_check_interval=0.2 \
    trainer.max_steps=50 \
    model.megatron_amp_O2=True \
    ++model.mcore_gpt=True \
    model.tensor_model_parallel_size=${TP_SIZE} \
    model.pipeline_model_parallel_size=${PP_SIZE} \
    model.micro_batch_size=1 \
    model.global_batch_size=32 \
    model.restore_from_path=${MODEL} \
    model.data.train_ds.file_names=${TRAIN_DS} \
    model.data.train_ds.concat_sampling_probabilities=[1.0] \
    model.data.validation_ds.file_names=${VALID_DS} \
    model.peft.peft_scheme=${SCHEME}

[NeMo W 2024-07-28 16:08:34 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    


[NeMo I 2024-07-28 16:08:34 megatron_gpt_finetuning:56] 
    
    ************** Experiment configuration ***********
[NeMo I 2024-07-28 16:08:34 megatron_gpt_finetuning:57] 
    name: megatron_gpt_peft_${model.peft.peft_scheme}_tuning
    trainer:
      devices: 1
      accelerator: gpu
      num_nodes: 1
      precision: bf16-mixed
      logger: false
      enable_checkpointing: false
      use_distributed_sampler: false
      max_epochs: 9999
      max_steps: 50
      log_every_n_steps: 10
      val_check_interval: 0.2
      gradient_clip_val: 1.0
    exp_manager:
      explicit_log_dir: ./results/Meta-llama3.1-8B-Instruct-titlegen
      exp_dir: ./results/Meta-llama3.1-8B-Instruct-titlegen
      name: ${name}
      create_wandb_logger: false
      wandb_logger_kwargs:
        project: null
        name: null
      resume_if_exists: true
      resume_ignore_no_checkpoint: true
      create_checkpoint_callback: true
      checkpoint_callback_params:
        monitor: validation_${mode

[NeMo W 2024-07-28 16:08:34 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/_graveyard/precision.py:49: The `MixedPrecisionPlugin` is deprecated. Use `pytorch_lightning.plugins.precision.MixedPrecision` instead.
    
GPU available: True (cuda), used: True


[NeMo I 2024-07-28 16:08:34 dist_ckpt_io:95] Using ('zarr', 1) dist-ckpt save strategy.


TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
[NeMo E 2024-07-28 16:08:34 exp_manager:703] exp_manager received explicit_log_dir: ./results/Meta-llama3.1-8B-Instruct-titlegen and at least one of exp_dir: ./results/Meta-llama3.1-8B-Instruct-titlegen, or version: None. Please note that exp_dir, name, and version will be ignored.
[NeMo W 2024-07-28 16:08:34 exp_manager:630] There were no checkpoints found in checkpoint_dir or no checkpoint folder at checkpoint_dir :results/Meta-llama3.1-8B-Instruct-titlegen/checkpoints. Training from scratch.


...

Metric val_loss improved by 0.016 >= min_delta = 0.001. New best score: 0.074
Epoch 0, global step 50: 'validation_loss' reached 0.07400 (best 0.07400), saving model to '/root/verb-workspace/results/Meta-llama3.1-8B-Instruct-titlegen/checkpoints/megatron_gpt_peft_lora_tuning--validation_loss=0.074-step=50-consumed_samples=1600.0.ckpt' as top 1


Epoch 0: : 100%|██████████| 50/50 [07:58<00:00, reduced_train_loss=0.0555, global_step=49.00, consumed_samples=1600.0, train_step_timing in s=5.660, val_loss=0.074] [NeMo I 2024-07-28 16:18:08 nlp_overrides:464] Removing checkpoint: /root/verb-workspace/results/Meta-llama3.1-8B-Instruct-titlegen/checkpoints/megatron_gpt_peft_lora_tuning--validation_loss=0.090-step=40-consumed_samples=1280.0.ckpt
[NeMo I 2024-07-28 16:18:08 nlp_overrides:464] Removing checkpoint: /root/verb-workspace/results/Meta-llama3.1-8B-Instruct-titlegen/checkpoints/megatron_gpt_peft_lora_tuning--validation_loss=0.090-step=40-consumed_samples=1280.0-last.ckpt


`Trainer.fit` stopped: `max_steps=50` reached.


Epoch 0: : 100%|██████████| 50/50 [07:59<00:00, reduced_train_loss=0.0555, global_step=49.00, consumed_samples=1600.0, train_step_timing in s=5.660, val_loss=0.074]


Restoring states from the checkpoint path at /root/verb-workspace/results/Meta-llama3.1-8B-Instruct-titlegen/checkpoints/megatron_gpt_peft_lora_tuning--validation_loss=0.074-step=50-consumed_samples=1600.0.ckpt
Restored all states from the checkpoint at /root/verb-workspace/results/Meta-llama3.1-8B-Instruct-titlegen/checkpoints/megatron_gpt_peft_lora_tuning--validation_loss=0.074-step=50-consumed_samples=1600.0.ckpt


In [49]:
 # Check that the LORA model file exists
!ls -l ./results/Meta-llama3.1-8B-Instruct-titlegen/checkpoints

total 307504
-rw-r--r-- 1 root root 146928174 Jul 28 16:18 'megatron_gpt_peft_lora_tuning--validation_loss=0.074-step=50-consumed_samples=1600.0-last.ckpt'
-rw-r--r-- 1 root root 146928174 Jul 28 16:18 'megatron_gpt_peft_lora_tuning--validation_loss=0.074-step=50-consumed_samples=1600.0.ckpt'
-rw-r--r-- 1 root root  21012480 Jul 28 16:18  megatron_gpt_peft_lora_tuning.nemo
